# Setup

In [40]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cluster import KMeans, DBSCAN

data: pd.DataFrame = pd.read_csv(
    filepath_or_buffer="../data_processing/cleaned_dataset.csv"
)

# Prepare Data

In [41]:
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80728 entries, 0 to 80727
Columns: 143 entries, Unnamed: 0.1 to track_genre_world-music
dtypes: float64(10), int64(133)
memory usage: 88.1 MB


,Unnamed: 0.1,Unnamed: 0,popularity,duration_s,explicit,danceability,energy,loudness,mode,speechiness,...,track_genre_spanish,track_genre_study,track_genre_swedish,track_genre_synth-pop,track_genre_tango,track_genre_techno,track_genre_trance,track_genre_trip-hop,track_genre_turkish,track_genre_world-music
0,93933,93933,0,141.506,0,0.3950,0.135,-17.867,0,0.0417,...,0,0,0,0,0,0,0,0,0,0
1,93638,93638,0,150.000,0,0.5390,0.207,-14.622,0,0.0518,...,0,0,0,0,0,0,0,0,0,0
2,93836,93836,0,197.786,0,0.3340,0.180,-14.943,0,0.0645,...,0,0,0,0,0,0,0,0,0,0
3,76134,76134,0,292.173,0,0.2790,0.273,-9.885,1,0.0320,...,0,0,0,0,0,0,0,0,0,0
4,16034,16034,0,283.933,0,0.0965,0.200,-16.970,1,0.0393,...,0,0,0,0,0,0,0,0,0,0


# Feature Selection and Engineering

In [59]:
cluster_df = data.copy().sample(n=1000)
cluster_df = cluster_df[
    [
        "duration_s",
        "explicit",
        "danceability",
        "energy",
        "loudness",
        "mode",
        "speechiness",
        "acousticness",
        "instrumentalness",
        "liveness",
        "valence",
        "tempo",
        # "key_A",
        # "key_A#",
        # "key_B",
        # "key_C",
        # "key_C#",
        # "key_D",
        # "key_D#",
        # "key_E",
        # "key_F",
        # "key_F#",
        # "key_G",
        # "key_G#",
        # "time_signature_3/4",
        # "time_signature_4/4",
        # "time_signature_5/4",
    ]
]

cluster_df.head()

,duration_s,explicit,danceability,energy,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
31872,209.658,1,0.404,0.585,-7.321,1,0.2950,0.102000,0.000000,0.056,0.2920,71.102
34011,211.852,0,0.684,0.589,-7.255,0,0.3530,0.235000,0.000005,0.400,0.4300,170.088
39061,165.746,1,0.695,0.564,-6.884,0,0.2580,0.099800,0.000033,0.455,0.2710,79.857
59882,312.133,0,0.469,0.270,-10.886,1,0.0283,0.827000,0.000000,0.687,0.2780,142.029
60060,264.827,0,0.418,0.917,-3.202,1,0.0396,0.000444,0.068000,0.102,0.0655,173.964


In [61]:
from sklearn.manifold import TSNE

def plot_tsne(df: pd.DataFrame, method: str):
    tsne: TSNE = TSNE(
        n_components=2,
        random_state=0,
        max_iter=1000,
        n_iter_without_progress=100,
        metric=method
    )
    tsne_df = tsne.fit_transform(df)

    sns.scatterplot(
        x=tsne_df[:, 0],
        y=tsne_df[:, 1],
        palette="deep",
        hue=df["duration_group"],
    )
    plt.show()

for method in ["cosine", "euclidean", "manhattan"]:
    plot_tsne(df=cluster_df, method=method)

KeyError: 'duration_group'